# PART B - With data Normalization

In [16]:
import pandas as pd
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [17]:
concrete_data = pd.read_csv('https://cocl.us/concrete_data')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [18]:
concrete_data.shape

(1030, 9)

In [19]:
concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [20]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

In [21]:
concrete_data_columns = concrete_data.columns

predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column

In [22]:
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [23]:
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

In [24]:
#Normalize the data
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


In [25]:
# number of predictors
n_cols = predictors_norm.shape[1]

In [26]:
# create model
model = Sequential()
model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
model.add(Dense(1))

# compile model
model.compile(optimizer='adam', loss='mean_squared_error')


In [27]:
#Create an numpy array to store the mean squared error for each of the 50 runs
iter_error=np.zeros(50)
#Iterate 50 times
for i in range(50):
    #make a new train/test split for each run
    X_train, X_test, y_train, y_test = train_test_split( predictors_norm, target, test_size=0.3, random_state=None)
    # fits the model on the train split this time with 100 epochs
    model.fit(X_train, y_train, validation_split=0.3, epochs=100, verbose=0)
    #Use the model created and generate the predicted outputs of the test split
    y_pred = model.predict(X_test)
    #Calculate the mean squared distance between the predicted value and the real value for the test split
    iter_error[i]=metrics.mean_squared_error(y_test, y_pred)
    print("Iteration " + str(i) + " had a mean squared error of " + str(iter_error[i]))


Iteration 0 had a mean squared error of 194.88938468468018
Iteration 1 had a mean squared error of 153.89494308199286
Iteration 2 had a mean squared error of 116.10460204918716
Iteration 3 had a mean squared error of 104.60814723699174
Iteration 4 had a mean squared error of 89.88864519102606
Iteration 5 had a mean squared error of 80.57807135507899
Iteration 6 had a mean squared error of 78.38666530219038
Iteration 7 had a mean squared error of 79.65889496391783
Iteration 8 had a mean squared error of 76.8841539299142
Iteration 9 had a mean squared error of 73.66630743572301
Iteration 10 had a mean squared error of 78.36225924566148
Iteration 11 had a mean squared error of 67.43318756638149
Iteration 12 had a mean squared error of 77.4636728059482
Iteration 13 had a mean squared error of 75.61198947523077
Iteration 14 had a mean squared error of 61.67672527137111
Iteration 15 had a mean squared error of 70.09349287838002
Iteration 16 had a mean squared error of 65.64166833426958
Itera

In [28]:
#reports the final results requested    
print("For the 50 runs the mean of the mean squared errors is " + str(np.mean(iter_error)) + " and the standard deviation is " + str(np.std(iter_error)))


For the 50 runs the mean of the mean squared errors is 69.71239858198807 and the standard deviation is 26.95683131553166


In [ ]:
# the error mean error is slightly higher than with Part B but the standard deviation is quite lower.